<a href="https://www.kaggle.com/code/syedasimalishah/approaching-from-back?scriptVersionId=107470720" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
sc = StandardScaler()
le = LabelEncoder()
ohe = OneHotEncoder()

In [ ]:
import os
os.listdir("/kaggle/input/spaceship-titanic")

In [ ]:
df1 = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")

In [ ]:
df1.head()

In [ ]:
df2 = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [ ]:
y = df1.iloc[:,-1]
df1 = df1.iloc[:,:-1]

In [ ]:
df1.shape,df2.shape

In [ ]:
final = pd.concat([df1,df2])

In [ ]:
import math
li1 = []
li2 = []
li3 = []
for a in final.Cabin:
  try:
    li1.append(a.split("/")[0])
    li2.append(a.split("/")[1])
    li3.append(a.split("/")[2])
  except AttributeError as e:
    li1.append(np.nan)
    li2.append(np.nan)
    li3.append(np.nan)

In [ ]:
dff = pd.DataFrame({"Deck":li1,"Num":li2,"Side":li3})

In [ ]:
lis1=[]
lis2=[]
for a in final.iloc[:,0]:
  lis1.append(int(a.split("_")[0]))
  lis2.append(int(a.split("_")[1]))

In [ ]:
dff2 = pd.DataFrame({"Group":lis1,"Number":lis2})

In [ ]:
final.drop([final.columns[3],final.columns[0]],axis=1,inplace=True)
final = final.iloc[:,:-1]
final

In [ ]:
final = pd.concat([final.reset_index(),dff,dff2],axis=1)

In [ ]:
final = final.iloc[:,1:]
final

In [ ]:
ptrain = final.iloc[:8693]
ptest = final.iloc[8693:]
ptrain["Transported"] = y

In [ ]:
ptrain.to_csv("ptrain.csv")
ptest.to_csv("ptest.csv")

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer
impute = SimpleImputer(strategy = "most_frequent")
knnimputer = KNNImputer(n_neighbors=25)

In [ ]:
final.head(1)

In [ ]:
f2 = final.drop(final.columns[[0,1,2,4,-5,-4,-3,-2]],axis=1)
f2

In [ ]:
f2 = impute.fit_transform(f2)
f2 = pd.DataFrame(f2)

In [ ]:
f2 = sc.fit_transform(f2)
f2 = pd.DataFrame(f2)

In [ ]:
f3 = pd.DataFrame(ohe.fit_transform(final.iloc[:,[0,1,2,4,-5,-3]]).toarray())

In [ ]:
cleaned = pd.concat([f2,f3],axis=1)
cleaned

In [ ]:
train = cleaned.iloc[:8693]
test = cleaned.iloc[8693:]
train.shape,test.shape

In [ ]:
def classifier(train,y,test):
  from sklearn.svm import SVC
  model = SVC(kernel="linear",random_state=0)
  model.fit(train,y)
  pred_1 = model.predict(test)
  
  from sklearn.linear_model import LogisticRegression
  model2 = LogisticRegression(random_state=0)
  model2.fit(train,y)
  pred_2 = model2.predict(test)

  from sklearn.tree import DecisionTreeClassifier
  model3 = DecisionTreeClassifier(criterion = "entropy",random_state=0)
  model3.fit(train,y)
  pred_3 = model3.predict(test)
  
  from sklearn.neighbors import KNeighborsClassifier
  model4 = KNeighborsClassifier(n_neighbors=5,metric="minkowski",p=2)
  model4.fit(train,y)
  pred_4 = model4.predict(test)
  
  test2 = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
  id = test2.iloc[:,0].values

  sub = pd.DataFrame({"PassengerId":id,"Transported":pred_1})
  sub.to_csv("Submit1.csv",index=False)
  
  sub = pd.DataFrame({"PassengerId":id,"Transported":pred_2})
  sub.to_csv("Submit2.csv",index=False)

  sub = pd.DataFrame({"PassengerId":id,"Transported":pred_3})
  sub.to_csv("Submit3.csv",index=False)
  
  sub = pd.DataFrame({"PassengerId":id,"Transported":pred_4})
  sub.to_csv("Submit4.csv",index=False)

  return pred_1,pred_2,pred_3,pred_4

In [ ]:
p,o,i,u = classifier(train,y,test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(p,o),accuracy_score(o,i),accuracy_score(i,u),accuracy_score(u,p)

In [ ]:
from sklearn.linear_model import LogisticRegression
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
'''model = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = loguniform(1e-5, 100)
# define search
search = RandomizedSearchCV(model, space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=cv, random_state=1)
# execute search
result = search.fit(train, y)'''

In [ ]:
result.best_params_,result.best_score_

In [ ]:
model = LogisticRegression(C= 0.048719032420512114, penalty= 'l2', solver= 'newton-cg')
model.fit(train,y)

In [ ]:
pred = model.predict(test)

In [ ]:
id = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv").iloc[:,0]

In [ ]:
sub = pd.DataFrame({"PassengerId":id,"Transported":pred})

In [ ]:
sub.to_csv("Submission.csv",index=False)

In [ ]:
pd.read_csv("Submission.csv")